**Probabilistic Matrix Factorization** 

PMF란 Factorization 기법 중 하나로써 데이터의 값이 Guassian distribution을 따른다고 가정하는 것이 특징이다. 즉, 추천 시스템에서는 점수값 R이 Gaussian 분포를 따른다고 가정하는 것이다. 시간 복잡도를 낮추기 위하여 m x n 형태의 점수 행렬을 d x m 모양의 U 행렬, 그리고 d x n 형태의 V행렬로 factorization 시키게 된다. U와 V 역시 각각 정규분포를 따르게 구성한다. 그렇게 되면 기존에 m x n개의 element를 학습시켜야 했던 반면, factorization 이후에는 d x (m + n)개의 element들만 학습시키면 된다. d는 하이퍼 파라미터이기에 상수이다. 즉, pmf의 시간 복잡도는 linearly 증가하게 된다. 따라서, sparse하고 imbalanced한 datasets에 효과적인 기법이다. 

알고리즘은 다음과 같다.

1. U, V 값을 d x m, 그리고 d x n 모양을 따르게 정규분포 값을 할당한다.
2. U의 전치행렬과 V를 곱하여 R의 예측 값을 구한다.
3. 실제 R값과 비교하고 역전파를 통해 U와 V를 학습시킨다.
4. 3을 정해진 epoch만큼 반복한다. 


In [ ]:
import pandas as pd
import torch

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ratings.csv')
ratings.describe()



rating_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
n_users, n_movies = rating_matrix.shape
# Scaling ratings to between 0 and 1, this helps our model by constraining predictions
min_rating, max_rating = ratings['rating'].min(), ratings['rating'].max()
rating_matrix = (rating_matrix - min_rating) / (max_rating - min_rating)


# Replacing missing ratings with -1 so we can filter them out later
rating_matrix[rating_matrix.isnull()] = -1
rating_matrix = torch.FloatTensor(rating_matrix.values)



# This is how we can define our feature matrices
# We're going to be training these, so we'll need gradients
latent_vectors = 5
user_features = torch.randn(n_users, latent_vectors, requires_grad=True)
user_features.data.mul_(0.01)
movie_features = torch.randn(n_movies, latent_vectors, requires_grad=True)
movie_features.data.mul_(0.01)


class PMFLoss(torch.nn.Module):
    def __init__(self, lam_u=0.3, lam_v=0.3):
        super().__init__()
        self.lam_u = lam_u
        self.lam_v = lam_v

    def forward(self, matrix, u_features, v_features):
        non_zero_mask = (matrix != -1).type(torch.FloatTensor)
        predicted = torch.sigmoid(torch.mm(u_features, v_features.t()))

        diff = (matrix - predicted) ** 2
        prediction_error = torch.sum(diff * non_zero_mask)

        u_regularization = self.lam_u * torch.sum(u_features.norm(dim=1))
        v_regularization = self.lam_v * torch.sum(v_features.norm(dim=1))

        return prediction_error + u_regularization + v_regularization

criterion = PMFLoss()
loss = criterion(rating_matrix, user_features, movie_features)

# Actual training loop now

latent_vectors = 30
user_features = torch.randn(n_users, latent_vectors, requires_grad=True)
user_features.data.mul_(0.01)
movie_features = torch.randn(n_movies, latent_vectors, requires_grad=True)
movie_features.data.mul_(0.01)

pmferror = PMFLoss(lam_u=0.05, lam_v=0.05)
optimizer = torch.optim.Adam([user_features, movie_features], lr=0.01)
for step, epoch in enumerate(range(1000)):
    optimizer.zero_grad()
    loss = pmferror(rating_matrix, user_features, movie_features)
    loss.backward()
    optimizer.step()
    if step % 50 == 0:
        print(f"Step {step}, {loss:.3f}")

# Checking if our model can reproduce the true user ratings
user_idx = 7
user_ratings = rating_matrix[user_idx, :]
true_ratings = user_ratings != -1
predictions = torch.sigmoid(torch.mm(user_features[user_idx, :].view(1, -1), movie_features.t()))
predicted_ratings = (predictions.squeeze()[true_ratings]*(max_rating - min_rating) + min_rating).round()
actual_ratings = (user_ratings[true_ratings]*(max_rating - min_rating) + min_rating).round()

print("Predictions: \n", predicted_ratings)
print("Truth: \n", actual_ratings)

Step 0, 8252.744
Step 50, 2600.260
Step 100, 1565.989
Step 150, 1126.650
Step 200, 940.850
Step 250, 844.803
Step 300, 786.246
Step 350, 745.761
Step 400, 715.755
Step 450, 692.681
Step 500, 674.285
Step 550, 659.440
Step 600, 646.908
Step 650, 636.440
Step 700, 627.467
Step 750, 619.645
Step 800, 612.851
Step 850, 606.842
Step 900, 601.589
Step 950, 596.938
Predictions: 
 tensor([4., 2., 4., 4., 3., 5., 3., 4., 5., 3., 3., 5., 2., 4., 4., 3., 4., 3.,
        3., 3., 5., 3., 3., 4., 3., 4., 3., 3., 5., 5., 3., 4., 5., 1., 3., 4.,
        3., 3., 2., 5., 3., 3., 3., 5., 3., 5., 3.], grad_fn=<RoundBackward0>)
Truth: 
 tensor([4., 2., 4., 4., 3., 5., 3., 4., 5., 3., 3., 4., 2., 5., 4., 3., 4., 3.,
        3., 3., 5., 3., 3., 4., 3., 5., 3., 3., 5., 5., 3., 4., 5., 1., 3., 4.,
        3., 4., 2., 5., 3., 3., 3., 5., 3., 4., 3.])


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks
